# 밑바닥부터시작하는딥러닝1

## 6.4 바른학습을 위해

### 6.4.1 오버피팅 Overfitting
- 신경망이 훈련 데이터에만 지나치게 적응되어 그 외의 데이터에는 제대로 대응하지 못하는 상태
- 매개변수가 많고 표현력이 높은 모델 / 훈련 데이터가 적음 => 오버피팅!

### 6.4.2 가중치 감소
- 학습과정에서 큰 가중치에 대해서는 그에 상응하는 큰 페널티를 부과하여 오버피팅 억제
- λ: 정규화의 세기를 조절하는 하이퍼파라미터

### 6.4.3 드롭아웃 Dropout
- 뉴런을 임의로 삭제하면서 학습하는 방법
- 훈련 때 은닉층의 뉴련을 무작위로 골라 삭제함

In [1]:
# self.mask: 훈련시에는 순전때마다 삭제할 뉴런을 False로 표시
# self.mask: x와 형상이 같은 배열을 무작위로 생성
# self.mask값이 dropout_ratio보다 큰 원소만 True로 설정
# 순전파때 신호를 통과시키는 뉴련은 역전파 때도 신호를 그대로 통과시키고 순전파 때 통과시키지 않은 뉴련은 역전파때도 신호 차단

class Dropout:
    
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None
        
    def forward(self, w, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)
        
    def backward(self, dout):
        return dout * self.mask

## 6.5 적절한 하이퍼파라미터 값 찾기

### 6.5.1 검증 데이터 Validation data
- 시험 데이터를 사용해서는 안되는 이유? 시험데이터를 사용하여 하이퍼파라미터 조정하면 하이퍼파라미터값이 시험 데이터에 오버피팅됨
- 그렇게 되면 다른 데이터에는 적응하지 못하니 범용 성능이 떨어지는 모델이 된다
- 따라서 조정용 데이터 사용 => 검증 데이터 (Validation data)
- 훈련데이터: 매개변수학습 / 검증데이터: 하이퍼파라미터성능평가 / 시험데이터: 신경망의 범용성능평가

In [8]:
import sys, os
import numpy as np
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist()

# 훈련 데이터를 뒤섞는다
def shuffle_dataset(x, t):
    permutation = np.random.permutation(x.shape[0])
    x = x[permutation,:] if x.ndim == 2 else x[permutation,:,:,:]
    t = t[permutation]
    return x, t

x_train, t_train = shuffle_dataset(x_train, t_train)

# 20%를 검증 데이터로 분할
validation_rate = 0.20
validation_num = int(x_train.shape[0] * validation_rate)

x_val = x_train[:validation_num]
t_val = t_train[:validation_num]
x_train = x_train[validation_num:]
t_train = t_train[validation_num:]

### 6.5.2 하이퍼파라미터 최적화
- 하이퍼파라미터의 '최적값'이 존재하는 범위를 조금씩 줄여나감

### 6.5.3 하이퍼파라미터 최적화 구현

In [9]:
# 가중치 감소 계수
weight_decay = 10**np.random.uniform(-8, -4)
lr = 10**np.random.uniform(-6, -2)

## 7.1 CNN 전체 구조
- 합성곱 계층 Convolutional layer

## 7.2 합성곱 계층

### 7.2.1 완전연결 계층의 문제점
- 완전연결 계층의 문제점: 데이터 형상이 무시된다
- 특징 맵(feature map): 합성곱 계층의 입출력 데이터
- 입력 특징 맵(input feature map): 합성곱 계층의 입력 데이터
- 출력 특징 맵(output feature map): 합성곱 계층의 출력 데이터

### 7.2.2 합성곱 연산
- 필터(커널) 연산: 합성곱 연산
- CNN에서는 필터의 매개변수 = 가중치

### 7.2.3 패딩 Padding
- 패딩: 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값으로 채움
- 주로 출력 크기를 조정할 목적으로 사용

### 7.2.4 스트라이드 stride
#### 입력크기: (H, W), 필터크기: (FH, FW), 출력크기: (OH, OW), 패딩 P, 스트라이드: S
### $OH={H+2P-FH\over S} +1$
### $OW={W+2P-FW\over S} +1$

### 7.2.5 3차원 데이터의 합성곱 연산
- 주의할 점: 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다
- 필터 자체의 크기는 원하는 값으로 설정 가능, 단 모든 채널의 필터가 같은 크기

### 7.2.6 블록으로 생각하기
- (C, FH, FW) = (채널수, 필터 높이, 필터 너비)
- 입력데이터 * 필터 = 출력데이터

$(C, H, W) * (C, FH, FW) = (1, OH, OW)$

- 연산의 출력으로 다수의 채널 내보내기, 필터를 FN 적용

$(C, H, W) * (FN, C, FH, FW) = (FN, OH, OW)$

- 편향 더해주기 (FN, 1, 1)

$(C, H, W) * (FN, C, FH, FW) =>  (FN, OH, OW) + (FN, 1, 1) => (FN, OH, OW)$

### 7.2.7 배치 처리
- 처리 효율 높임
- 4차원 데이터로 저장 (데이터 수, 채널 수, 높이, 너비)
- 아래는 데이터가 N개일 때 배치 처리

$(N, C, H, W) * (FN, C, FH, FW) =>  (N, FN, OH, OW) + (FN, 1, 1) => (N, FN, OH, OW)$

## 7.3 풀링 계층 Pooling
- 학습해야 할 매개변수가 없다
- 채널 수가 변하지 않는다
- 입력의 변화에 영향을 적게 받는다 (강건하다)

## 7.4 합성곱/풀링 계층 구현하기

### 7.4.1 4차원 배열

In [25]:
x = np.random.rand(10, 1, 28, 28)
print("x.shape:", x.shape)
print("x[0].shape:", x[0].shape)
print("x[1].shape:", x[1].shape)

x.shape: (10, 1, 28, 28)
x[0].shape: (1, 28, 28)
x[1].shape: (1, 28, 28)


### 7.4.2 im2col로 데이터 전개
- CNN은 4차원배열로 저장, 2차원 출력 데이터 -> 4차원으로 변형

### 7.4.3 합성곱 계층 구현하기
##### im2col(input_data, filter_hm filter_w, stride=1, pad=0)

- input_data: (데이터수, 채널수, 높이, 너비)
- filter_h: 필터의 높이
- filter_w: 필터의 너비
- stride: 스트라이드
- pad: 패딩

In [ ]:
im2col(input_data, filter_hm filter_w, stride=1, pad=0)
# input_data: (데이터수, 채널수, 높이, 너비)
# filter_h: 필터의 높이
# filter_w: 필터의 너비
# stride: 스트라이드
# pad: 패딩

In [3]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.util import im2col

x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [ ]:
class Convolution:
    def __init__(self, w, b, stride=1, pad=0):
        self.w = w
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.w.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH)/self.stride)
        out_w = 1 + int((W + 2*self.pad - FW)/self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_w) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

### 7.4.4 풀링 계층 구현하기
1. 입력 데이터 전개
2. 행병 최댓값 구하기
3. 적절한 모양으로 성형

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h)/self.stride)
        out_w = int(1 + (W - self.pool_w)/self,stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

        return out

## 7.5 CNN 구현하기
- input_dim: 입력 데이터(채널수, 높이, 너비)의 차원
- conv_param: 합성곱 계층의 하이퍼파라미터
- hidden_size: 은닉층의 뉴런 수
- output_size: 출력층의 뉴런 수
- weight_init_std: 초기화 때의 가중치 표준편차